## PART 0: Reviewing Earlier Steps



As almost all the "useful" journals are identified for Q1 100K run, it is necessary to understand & review the Scimago datasets and the earlier functions. As it was seen in the previous steps, there are different versions (both online & offline), (two journals merged into one after a certain year) of some journals, it important to create a final eligible journals dataset to be merged with the date results. 

Another factor is that, during complete_one runs there are 30K+ articles that have completed the complete pipeline and ready to be examined. These articles are currenty stored in date_df & date_df_second_run datasets. These journals must be utilized for maximum efficiency and minimizing unnecessary time loss. 

In [1]:
import pandas as pd
import pickle 
import numpy as np

#from quartile import *


#from scihub_upgraded import SciHub
#from crossref.restful import Works, Etiquette

#import warnings
#warnings.filterwarnings("ignore")

After analysis, it is seen that:

* scimago.py -> is responsible for "cat_sbj_dict" that is used in quartile.py for "Category" to "Subj. Area" matching.

* quartile.py -> is responsible for transforming Scimago excels into Q1-4 journal datasets. It also splits the journals based on their "highest Q'd" subj. areas, and splits their Total Docs. accordingly. As the remainder is distributed randomly, each run of this file generates slightly different doc. count for journals with "multimode"d subj areas. However, this should not effect the results greatly. The code also looks OK and efficient except for an .iterrows() loop, which can be improved but is ignored as it is not a huge performance or time issue.

* As Unpy is removed from the overall pipeline, all Unpy related code is also shelved.

* scimago_upgraded -> is responsible for the whole get_dates loop as the entire PDF & date extraction is now executed via SciHub.


The biggest improvement area while we are waiting for Q1 useful journals to be completed are making sure that the final journal list is compatible with quartile.py flow.





In [7]:
%store -r retr_complete_one

In [3]:
retr_complete_one.Results.map(type).value_counts()

<class 'list'>     60596
<class 'float'>    10302
<class 'str'>       3523
Name: Results, dtype: int64

In [4]:
retr_complete_one[retr_complete_one.Results.map(type)==list].Results.map(len).value_counts()

3    31022
2    29574
Name: Results, dtype: int64

In [5]:
retr_complete_one.head()

,Issn,Year,Total_Docs,Sample_Count,CrossRef_retr,DOI,Results
0,[00011541],2010,294,1,True,10.1002/aic.12400,[https://sci.bban.top/pdf/10.1002/aic.12400.pd...
1,[00011541],2011,315,1,True,10.1002/aic.12671,[https://sci.bban.top/pdf/10.1002/aic.12671.pd...
2,[00011541],2012,347,1,True,10.1002/aic.13810,[https://sci.bban.top/pdf/10.1002/aic.13810.pd...
3,[00011541],2013,422,1,True,10.1002/aic.14056,[https://sci.bban.top/pdf/10.1002/aic.14056.pd...
4,[00011541],2014,359,1,True,10.1002/aic.14601,[https://sci.bban.top/pdf/10.1002/aic.14601.pd...


In [2]:
# Get q_df(s):
q_df = quar_df_creator(1)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [4]:
# Create q_jrnl & q_pivor using q_df & article count target:

q_jrnl, q_pivot = retr_ready(q_df,100000)

In [5]:
q_jrnl

,Issn,Year,Sbj_Area,Total_Docs
0,"[00346861, 15390756]",2010,Physics and Astronomy,74
1,"[15454509, 00664154]",2010,"Biochemistry, Genetics and Molecular Biology",28
2,"[10614036, 15461718]",2010,"Biochemistry, Genetics and Molecular Biology",248
3,"[15424863, 00079235]",2010,Medicine,36
4,"[00928674, 10974172]",2010,"Biochemistry, Genetics and Molecular Biology",471
...,...,...,...,...
95486,[00083968],2020,Arts and Humanities,18
95487,[00732435],2020,Social Sciences,10
95488,[00732435],2020,Arts and Humanities,9
95489,"[10601503, 14701553]",2020,Social Sciences,16


In [12]:
q_ready = q_jrnl.groupby([q_jrnl["Issn"].map(tuple),q_jrnl["Year"]])["Total_Docs"].sum().reset_index()


In [13]:
q_ready

,Issn,Year,Total_Docs
0,"(00011541, 15475905)",2010,294
1,"(00011541, 15475905)",2011,315
2,"(00011541, 15475905)",2012,347
3,"(00011541, 15475905)",2013,422
4,"(00011541, 15475905)",2014,359
...,...,...,...
74416,"(8756758X, 14602695)",2017,170
74417,"(8756758X, 14602695)",2018,199
74418,"(8756758X, 14602695)",2019,210
74419,"(8756758X, 14602695)",2020,221


In [8]:
retr_complete_one

,Issn,Year,Total_Docs,Sample_Count,CrossRef_retr,DOI,Results
0,[00011541],2010,294,1,True,10.1002/aic.12400,[https://sci.bban.top/pdf/10.1002/aic.12400.pd...
1,[00011541],2011,315,1,True,10.1002/aic.12671,[https://sci.bban.top/pdf/10.1002/aic.12671.pd...
2,[00011541],2012,347,1,True,10.1002/aic.13810,[https://sci.bban.top/pdf/10.1002/aic.13810.pd...
3,[00011541],2013,422,1,True,10.1002/aic.14056,[https://sci.bban.top/pdf/10.1002/aic.14056.pd...
4,[00011541],2014,359,1,True,10.1002/aic.14601,[https://sci.bban.top/pdf/10.1002/aic.14601.pd...
...,...,...,...,...,...,...,...
74416,"[8756758X, 14602695]",2017,170,1,True,10.1111/ffe.12617,[https://sci.bban.top/pdf/10.1111/ffe.12617.pd...
74417,"[8756758X, 14602695]",2018,199,1,True,10.1111/ffe.12803,[https://sci.bban.top/pdf/10.1111/ffe.12803.pd...
74418,"[8756758X, 14602695]",2019,210,1,True,10.1111/ffe.13083,[https://sci.bban.top/pdf/10.1111/ffe.13083.pd...
74419,"[8756758X, 14602695]",2020,221,1,True,10.1111/ffe.13260,[https://sci.bban.top/pdf/10.1111/ffe.13260.pd...


The only USEFUL finding in this step is that, retr_complete_one has a slightly altered Issn column with fewer ISSN codes than a newly created version of it (q_ready).

As a result, this column will be REPLACED once all the work in the previous nb is completed!

## PART 1: Creating a COMPELETE (1st + 2nd run) date_df

In [4]:
%store -r date_df
%store -r second_run_date_df

In [24]:
duplicate_dois = set(date_df.index.tolist()).intersection(set(second_run_date_df.index.tolist()))
date_df.drop(duplicate_dois, inplace=True)

date_df_full = date_df.append(second_run_date_df,verify_integrity=True)
date_df_full.drop("Year", axis=1, inplace=True)

date_df_full is created!

In [243]:
%store date_df_full

Stored 'date_df_full' (DataFrame)


In [244]:
with open("date_df_full","wb") as fp:
    pickle.dump(date_df_full, fp)

The analysis below shows that, the missing (.) in the regex results in a 0.8% percent found date increase. This alone is not necessary for re-doing the entire dataset BUT:

* we now have the direct URLs for the successful articles, which should quicken & simplify the process if we decide to re-run
* KEYWORDS extraction might be necessary for future anaylsis, which would give us a second chance to loop through anyways!

In [46]:
sh = SciHub()
new_date_regex_test = date_df_full[date_df_full.Results.map(len)==2].sample(1000)
new_date_regex_test["New_Results"] = new_date_regex_test.index.map(sh.get_dates)
new_date_regex_test["New_Results"].map(len).value_counts()

2     990
3       8
21      1
13      1
Name: New_Results, dtype: int64

### PART 1.A: date_df_full_analysis

TTD for date_df_full:

* "skipped"  -> not filtered BUT in date_df_str_fail DONE(!)
* "published" column function -> DONE!
* "Results" column expand function -> DONE!
* "language" analysis (can we extact the lang. from the title for non-en articles) -> can't extract dates if not in Eng. so will be DROPPED & DONE!
* Check "ISSN" format 
* Simplify "author" column -> maybe create a author_df -> DONE!

In [242]:
date_df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128374 entries, 10.1002/aic.12400 to 10.1177/0096340214539115
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   reference-count         128374 non-null  int64 
 1   publisher               128371 non-null  object
 2   published-print         120056 non-null  object
 3   is-referenced-by-count  128374 non-null  int64 
 4   title                   128374 non-null  object
 5   author                  120278 non-null  object
 6   published-online        82614 non-null   object
 7   reference               89462 non-null   object
 8   container-title         128374 non-null  object
 9   language                57136 non-null   object
 10  issued                  128374 non-null  object
 11  ISSN                    128374 non-null  object
 12  subject                 128013 non-null  object
 13  published               62215 non-null   object
 14  Results

In [95]:
# 1- "not_in Scihub" FIX (date_df_full):

date_df_full.loc[date_df_full.Results.map(type)==str, "Results"] = date_df_full[date_df_full.Results.map(type)==str]["Results"].str.replace("article_not_in Scihub","article_not_in_Scihub")

# STORED & DUMPED AFTER THIS

date_df_full[date_df_full.Results.map(type)==str].Results.value_counts()

article_not_in_Scihub    9485
skipped                  3868
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
Name: Results, dtype: int64

In [96]:
date_df_full[date_df_full.Results.map(type)==list].Results.map(len).value_counts()

2    78218
3    36495
Name: Results, dtype: int64

In [231]:
# 2- SPLITTING THE DATASETS (date_df_ok):

date_df_str_fail = date_df_full[date_df_full.Results.map(type)==str].copy()

date_df_ok = date_df_full[date_df_full.Results.map(type)==list].copy()

In [232]:
# 3- "RESULTS" COLUMN ENGINEERING:
def result_mapper(lst):
    if len(lst) == 2:
        return lst[0], np.nan, np.nan
    elif len(lst) == 3:
        return lst[0], lst[1], lst[2]

date_df_ok["direct_url"], date_df_ok["Rec_date"], date_df_ok["Acc_date"] = zip(*date_df_ok.Results.map(result_mapper))

In [233]:
date_df_ok["direct_url"].map(type).value_counts()

<class 'str'>    114759
Name: direct_url, dtype: int64

In [234]:
date_df_ok["Rec_date"].map(type).value_counts()

<class 'float'>       78245
<class 'str'>         34050
<class 'NoneType'>     2464
Name: Rec_date, dtype: int64

In [235]:
date_df_ok["Acc_date"].map(type).value_counts()

<class 'float'>       78245
<class 'str'>         30891
<class 'NoneType'>     5623
Name: Acc_date, dtype: int64

In [236]:
# 4- "PUBLISHED" COLUMN ENGINEERING:

date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].isna()) & (date_df_ok["published-online"].notna()),"published"] = date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].isna()) & (date_df_ok["published-online"].notna()),"published-online"]
date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].notna()) & (date_df_ok["published-online"].isna()),"published"] = date_df_ok.loc[(date_df_ok.published.isna()) & (date_df_ok["published-print"].notna()) & (date_df_ok["published-online"].isna()),"published-print"]

def earlier_date_part(row):
    pp_year = row["published-print"]["date-parts"][0][0]
    po_year = row["published-online"]["date-parts"][0][0]
       
    if pp_year<po_year:
        return row["published-print"]
    elif po_year<pp_year:
        return row["published-online"]
    else:
        try:
            pp_month = row["published-print"]["date-parts"][0][1]
        except IndexError:
            pp_month = 12
        try:
            po_month = row["published-online"]["date-parts"][0][1]
        except IndexError:
            po_month = 12
        if pp_month<po_month:
            return row["published-print"]
        elif po_month<pp_month:
            return row["published-online"]
        else:
            return row["published-online"]


date_df_ok.loc[date_df_ok.published.isna(),"published"] = date_df_ok[date_df_ok.published.isna()].apply(earlier_date_part, axis=1)

In [237]:
# 5- "REFERENCE" & "LANGUAGE" COLUMN ANALYSIS:

# It seems that both columns are not useful, and will be dropped. They can be retrieved from date_df_full in the future if necessary.

date_df_ok.drop(["reference","language"], axis=1, inplace=True)

In [238]:
# 6- "AUTHOR" COLUMN ANALYSIS:

date_df_ok.loc[date_df_ok.author.notna(),"first_author"] = date_df_ok[date_df_ok.author.notna()].author.map(lambda x: x[0])

In [240]:
%store date_df_ok

Stored 'date_df_ok' (DataFrame)


In [241]:
with open("date_df_ok","wb") as fp:
    pickle.dump(date_df_ok, fp)

### PART 1.B: retr_complete_one & second_run_df analysis

In [19]:
%store -r retr_complete_one
%store -r second_run_df

In [20]:
retr_complete_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74421 entries, 0 to 74420
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Issn           74421 non-null  object
 1   Year           74421 non-null  int32 
 2   Total_Docs     74421 non-null  int64 
 3   Sample_Count   74421 non-null  int64 
 4   CrossRef_retr  74421 non-null  object
 5   DOI            74421 non-null  object
 6   Results        64119 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 4.3+ MB


There are 3 dataframes to work on:

* no need for DOI info in any of them
* no need for "Sample_Count"

1- second_run_df:
* replace Second_CR_retr with CrossRef_retr
* get True & False from "2nd_Results"

2- retr_compete_one:
* drop all included in other 2
* get True & False from "Results" 

3- missing49:
* loop through CR + SH get_dates

In [ ]:
missing49 = retr_complete_one[(retr_complete_one.CrossRef_retr == True) & (retr_complete_one.Results.map(type)==float)].copy()

In [136]:
# 1- second_run_df:

second_run_df["SH_Results"] = second_run_df["2nd_Results"].map(lambda x: True if len(x)>0 else False)
second_run_df.drop(["Sample_Count","CrossRef_retr","DOI","Results","2nd_Results"], axis=1, inplace=True)
second_run_df.rename(columns={"Second_CR_retr":"CR_Results"},inplace=True)

In [ ]:
# 2- retr_complete_one:
retr_complete_one.drop(second_run_df.index.tolist(), inplace=True)
retr_complete_one.drop(missing49.index.tolist(), inplace=True)
retr_complete_one["SH_Results"] = True
retr_complete_one.drop(["Sample_Count","DOI","Results"], axis=1, inplace=True)
retr_complete_one.rename(columns={"CrossRef_retr":"CR_Results"},inplace=True)

In [ ]:
# 3- missing49:
missing49.drop(["Sample_Count","DOI","Results"], axis=1, inplace=True)
missing49.rename(columns={"CrossRef_retr":"CR_Results"}, inplace=True)

In [164]:
# 3- missing49 cont.:

from scihub_upgraded import SciHub
from crossref.restful import Works, Etiquette

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', 'oguzkokes@gmail.com')
ogi_works = Works(etiquette=my_etiquette)

sh = SciHub()

# The full loop code for missing49:

issn_rry = missing49.Issn.values
year_rry = missing49.Year.values

#second_run_df.reset_index(drop=True, inplace=True)

miss49_meta_list = []
miss49_sh_date_dict = {}
useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'reference', 'container-title', 'issued', 'ISSN', 'subject']

for i in range(len(missing49)):
    issn = issn_rry[i]
    year = year_rry[i]
    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(1).select(useful_cols)] 

    if len(loop_meta) == 0:
        missing49.loc[missing49.index[i],"Second_CR_retr"] = False
    else:
        missing49.loc[missing49.index[i],"Second_CR_retr"] = True
        missing49.loc[missing49.index[i],"DOI"] = loop_meta[0]["DOI"]
        miss49_sh_date_dict[loop_meta[0]["DOI"]] = sh.get_dates(loop_meta[0]["DOI"])

    miss49_meta_list.extend(loop_meta)



In [176]:
# 3- missing49 cont.:

missing49["Results"] = missing49.DOI.map(miss49_sh_date_dict)
missing49["SH_Results"] = missing49.Results.map(len) == 3
missing49.drop(["Second_CR_retr","DOI","Results"], axis=1, inplace=True)

49

In [187]:
# Concat all 3 journal related datasets: retr_onetwo

retr_onetwo = pd.concat([second_run_df,retr_complete_one, missing49], verify_integrity=True)

In [189]:
%store retr_onetwo

Stored 'retr_onetwo' (DataFrame)


In [245]:
with open("retr_onetwo","wb") as fp:
    pickle.dump(retr_onetwo, fp)

### PART 1.C: Add articles from missing49 to date_df_full & date_df_str_fail & date_df_ok

In [192]:
miss49_df = pd.DataFrame(miss49_meta_list)
miss49_df.set_index("DOI",drop=True,inplace=True)
miss49_df["Results"] = miss49_df.index.map(miss49_sh_date_dict)
miss49_df.rename(columns={"references-count":"reference-count"}, inplace=True)

date_df_full = pd.concat([date_df_full,miss49_df],verify_integrity=True)

In [ ]:
#Rather than looping again, date_df_ok & date_df_str_fail will be created again from date_df_full by going through the steps in the previous chapter. All datasets are SAVED & STORED again afterwards.

date_df_ok

retr_one_two

### PART 2: ...